In [1]:
import sys
sys.path.append('../')
import copy
import math
import numpy as np
from utils.controller_utils import Robot
import rospy
import tools.rotations as rot
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')

%load_ext autoreload
%autoreload 2

r = Robot(optitrack_frame_names=['iiwa_base7', 'realsense_m'], position_control=False)

target_pose = np.copy(r.q)
target_pose[0] -= 0.5
print(target_pose)
r._iiwa_joint_control(target_pose)


Kinematic model has been loaded from ../kinematics/q2pose_left.txt


Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_0']/collision[1]
Unknown tag "self_collision_checking" in /robot[@name='iiwa7']/link[@name='iiwa_link_0']
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_1']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_2']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_3']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_4']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_5']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_6']/collision[1]
Unknown tag "material" in /robot[@name='iiwa7']/link[@name='iiwa_link_7']/collision[1]
Unknown tag "hardwareInterface" in /robot[@name='iiwa7']/transmission[@name='iiwa_tran_1']/actuator[@name='iiwa_motor_1']
Unknown tag "robotNamespace" in /robot[@name='iiwa7']/transmission[@name='iiwa_tran_1']
Unkno

[-0.82954659  0.51365568 -0.18168373 -0.98127463  0.01021363  1.18886291
  1.04163815]
Linear interpolation by 1000 joints
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
e

array([-0.26019622,  0.54607259, -0.30892803, -0.85208055, -0.00926371,
        1.29349952,  1.08663196])

In [ ]:
#pre-defined pose
home_pose = np.array([ 0.03063398, -0.15441399,  0.82104033,  0.82512679,  0.00664029,
       -0.2039733 ,  0.80409743,  0.        , -0.10271061, -0.06070025,
        0.90650362,  0.81377007,  0.86300686,  0.44554398,  0.08874171,
        0.77574977])

catch_pose = np.array([-0.50889807,  1.74874918,  0.12737114,  0.39226041,  0.07008444,
        1.64808905,  1.36754356,  0.21940653,  0.0937576 ,  1.77053021,
        1.36580765,  0.55991777,  0.94150295,  0.14131779, -0.24060529,
        0.96755097])

envelop_pose = np.array([ 0.07941064,  1.77857954,  0.44877399,  0.62444235,  0.29191241,
        1.55038098,  0.61655537,  0.36046541,  0.4311105 ,  1.76264807,
        1.02352552,  0.9226552 ,  1.54144452,  0.48900141, -0.08175917,
        1.3566044 ])

while True:
    r.move_to_joints(home_pose,vel=[0.2,8.0])
    rospy.sleep(2)
    r.move_to_joints(envelop_pose, vel=[0.2,8.0])
    rospy.sleep(2)

In [ ]:
# position control
r.iiwa_go_home()

In [ ]:
# impedance control
target_pose = r.x
target_pose[0] -=0.2
target_pose[2] -=0.2
efforts = []
r.iiwa_cartesion_impedance_control(target_pose,vel=1.0)


In [ ]:
# move along a circle under impedance control
def generate_circle_via_points(current_pose, radius=0.05, num_points=10):
    x0, y0, z0 = current_pose[:3]
    center_x = x0 - radius
    center_y = y0
    center_z = z0

    theta_array = np.linspace(0, 2*math.pi, num_points)

    via_points = np.zeros((num_points, 7))
    for i, theta in enumerate(theta_array):
        # x = center_x + radius*cos(theta), y = center_y + radius*sin(theta)
        x = center_x + radius * math.cos(theta)
        y = center_y + radius * math.sin(theta)
        z = center_z
        via_points[i, :3] = np.array([x, y, z])

        via_points[i, 3:] = current_pose[3:]
    return via_points

current_pose = r.x
via_points = generate_circle_via_points(current_pose, radius=0.2, num_points=20)

r.iiwa_cartesion_impedance_control(via_points)

In [3]:
target_pose = np.copy(r.x)
target_pose[2] -= 0.2
q = r.trac_ik_solver(target_pose)
# r.move_to_joints(q, vel=[0.1, 1.0])
r.move_to_target_cartesian_pose(target_pose)
print(r.q - q)

Linear interpolation by 906 joints
Trajectory has been executed.
[ 0.27561274 -0.44333757 -0.34632029  0.0697855   0.35094933  0.25397638
  0.42520985]
